In [1]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from jax.scipy.optimize import minimize 
from LombScargle import periodogram, psd
from hypothesis_testing.bayes_factor import logB
from quasars import prep

from simulations.util import  *

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
id = 19827
#id = 253736
time, mag, mag_err, freq, prior_params = prep.prepare_data(id)
print(len(time))

score_white, amps = jax.vmap(periodogram.lomb_scargle(time, mag, True, mag_err))(freq)
ibest = jnp.argmax(score_white)
score_white, amp = score_white[ibest], amps[ibest]
print(score_white)
freq_best = freq[ibest]
print(1./freq_best)
tshift = 0
plt.title('id: ' + str(id))
plt.errorbar(time-tshift, mag, mag_err, fmt = '.', capsize = 1.3, color= 'black')
t = jnp.linspace(jnp.min(time), jnp.max(time), 1000)
model= periodogram.fit(t, freq_best, amp)
plt.plot(t-tshift, model, color = 'teal')
plt.gca().invert_yaxis()

plt.show()

ValueError: zero-size array to reduction operation max which has no identity

In [19]:
from quasars import prior

def signal(key, time, generator_logfreq):
    key1, key2 = jax.random.split(key)
    phase = jax.random.uniform(key1) * 2 * jnp.pi
    freq_injected = jnp.exp(generator_logfreq(key2))
    return jnp.sin(2 * jnp.pi * freq_injected * time + phase), 1./freq_injected

key = jax.random.PRNGKey(42)
id = 19827
#id = 253736
time, _, mag_err, freq, prior_params = prep.prepare_data(id)
nlogpr_logfreq, generator_logfreq = prior.uniform_with_smooth_edge(*prior_params)

mag, period_injected = signal(key, time, generator_logfreq)

In [3]:
known = pd.read_csv('quasars/known.csv')
data= pd.read_csv('data/data.csv')
data= data.sort_values('myid')
period = np.array(data['period'])
period0 = np.array(known['period'])

print(period[13], period0[13])

654.4673 309.7


In [18]:
# setup
key = jax.random.PRNGKey(42)

scratch = '/pscratch/sd/j/jrobnik/quasars/'
id = jnp.load(scratch + 'ids.npy')# a random id
print(len(id))
#id = jnp.array(pd.read_csv('quasars/known.csv')['id'], dtype= int)[4]

#hyp = jnp.array([0.1, 120.])
    
df = pd.read_csv(scratch + str(id) + '.csv')
time = jnp.array(df['time'])
mag = jnp.array(df['mag'])
mag_err = jnp.array(df['mag_err'])
#key_time, key_data = jax.random.split(key)
#time= jax.random.uniform(key_time, shape = (1000,)) * 2000
#mag_err= jnp.ones(time.shape) * 0.0

#cov = psd.covariance(time, psd.drw_kernel(*hyp), mag_err)
T = jnp.max(time) - jnp.min(time)
fmin, fmax = 1./T, 1./60.
freq = jnp.logspace(jnp.log10(fmin), jnp.log10(fmax), 1000)
#data = irregular_spaced(key, cov)

#print(logB(time, data, mag_err, freq))